In [2]:
# FRI domain
from field import find_generator, check_generator, get_primitive_root
from utils import is_prime

# Find L
n = 16
p = 337

assert is_prime(p), f'{p} is not prime'

# Find generator
g = find_generator(p)
assert g != None, "Generator not found"
print(f'generator g = {g}')

# Check order of the group generated by g
check_generator(g, p)

# Primitive Nth root
w = get_primitive_root(g, p, n)

# Evaluation domain L
L = [pow(w, i, p) for i in range(0, n)]
print(f'L = {L}')

generator g = 10
L = [1, 191, 85, 59, 148, 297, 111, 307, 336, 146, 252, 278, 189, 40, 226, 30]


In [ ]:
# A Walk-Through of a Simple zk-STARK Proof

# N = 4
# A = [1, 0, 1, 1]
# g = 4 mod 17
# f(g[i]) = A[i] for i < N
# A[i] = 0 or 1 -> A[i](A[i] - 1) = 0 for i < N -> f(g[i])^2 - f(g[i]) = 0
# c(x) = f(x)^2 - f(x)
# u(x) = ((x-g[0])(x - g[1])...(x - g[N - 1])) = x^N - 1
# p(x) = c(x) / u(x)
# p(x) is polynomial of degree <= deg(c) - N <-> c(x) = 0 for x = g[i] <-> All A[i] = 0 or 1

# Query
# Verifier checks p(x)u(x) = c(x) (query x in L - G, query x in G breaks zero knowledge)
# 1. Verifier queries for z in L - G
# 2. Prover sends p(z) and f(z) and Merkle proofs of p(z) and f(z)
# 3. Verifier checks p(z)u(z) = c(z) and Merkle proofs

# num queries for high probability of fraud detection
# poly degree = d = 5
# evaluation domain L = 100
# num of dishonest evaluation = |L| - d = 95
# probability of getting caught after k queries = 1 - probability of not getting caught after k queries ~ 1 - (d / |L|)^k

# TODO - cheat if p is not a low degree
# TODO - example of false trace?

# Apply FRI to p

In [3]:
import numpy as np
from fft import fft, ifft, eval_poly
from field import F
from polynomial import Polynomial
import merkle

# Trace
T = [1, 0, 1, 1]

# F[P] = prime field
P = 17
M = len(T)
# Generator
g = 4
# Subgroup of order M generated by g
G = [pow(g, i, P) for i in range(M)]
print("G", G)
assert len(set(G)) == M

# Trace polynomial f(G[i]) = A[i]
f = Polynomial(ifft(T, G, P), lambda x: F(x, P))
assert f(G) == T

# G < L <= F[P] - {0}
# |L| = N
N = 16
assert M < N

# Primitive Nth root
w = 3
assert pow(w, N, P) == 1
assert pow(w, N // 2, P) == (-1 % P)

L = [pow(w, i, P) for i in range(P - 1)]
print("L", L)
assert len(set(L)) == N

# Constraint polynomial c(x) = f(x)^2 - f(x) = A[i](A[i] - 1) = 0 for G[i]
c = f*f - f

# Composition polynomial (RISCZERO calls in validity polynomial) 
# z(x) = (x - g^0)(x - g^1)...(x - g^(M-1)) = x^M - 1
# q(x) = c(x) / z(x)

# Why divide c(x) by z(x)? 
# Check c(x) -> need to check c(g^i) = 0 for 0 <= i < M
# c(g^i) = 0 for 0 <= i < M <-> g^i is a root of c <-> z divides c (c(x) / z(x) = 0 mod P)
# Dividing by z, we only do 1 check instead of M checks

# NOTE: in production several constraints are combined pseudo randomly with a1, a2, ...
# a1 * q1(x) + a2 * q2(x) + ... for constraints q1(x), q2(x), ...

# z(x) = x^M - 1
z = [0 for _ in range(M + 1)]
z[0] = -1 % P
z[-1] = 1
z = Polynomial(z, lambda x: F(x, P))

q = c / z
# Check q(G[i]) != 0
assert all(y != 0 for y in q(G))

# Prover sends Merkle root of q(x) and f(x)
hashes_f = [merkle.hash_leaf(str(y)) for y in f(L)]
hashes_q = [merkle.hash_leaf(str(y)) for y in q(L)]
root_f = merkle.commit(hashes_f)
root_q = merkle.commit(hashes_q)

# Verifier checks q(x)z(x) = c(x) for random x, x not in G
NUM_QUERIES = 5
assert NUM_QUERIES <= N - M
# X = {(index of x, x) for x in L - G}
# Q = Random selections without replacement from X
X = [(i, v) for i, v in enumerate(L) if v not in G]
Q = [X[i] for i in np.random.choice(range(len(X)), size = NUM_QUERIES, replace = False)]

for (i, x) in Q:
    # Prover sends f(x), q(x) and Merkle proofs
    fx = f(x)
    qx = q(x)
    proof_f = merkle.open(hashes_f, i)
    proof_q = merkle.open(hashes_q, i)

    # Verifier checks q(x)z(x) = c(x) and Merkle proofs
    cx = fx * fx - fx
    zx = z(x)
    assert qx * zx == cx

    assert merkle.verify(proof_f, root_f, hashes_f[i], i)
    assert merkle.verify(proof_q, root_q, hashes_q[i], i)

# Low degree testing - Verifier checks q(x) is low degree
# TODO - offset for q(L)?
# TODO - use same merkle root as proof_q?
print("q =", q)
print("degree of q =", q.degree())

G [1, 4, 16, 13]
L [1, 3, 9, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6]
q = [14, 8, 1]
degree of q = 2


In [3]:
import numpy as np
import random
import fri
from field import F
import polynomial
from polynomial import Polynomial
import merkle
import iop
from utils import is_pow2, is_prime

### Setup ###
# FRI domain L
# message (M) -> poly degree <= M - 1 -> RS code (N) <= |L| size of evaluation domain
# N = |L| = 2**K
# L = [1, w, w^2, ..., w^(N - 1)], w is Nth root of unity
# prime field F_p, |F_p| > |L|, N divides p - 1 (needed for finding primitive root of unity)

# Lenght of message
M = 4
ys = [random.randint(0, 1) for _ in range(M)]
print("ys:", ys)

# Expansion factor from message length M to RS code length N
# exp_factor * M = N
EXP_FACTOR = 4

# P = 1 + 407 * (1 << 119)
# Prime
P = 337
# Length of RS code
N = M * EXP_FACTOR
# Nth primitive root
w = 191

# assert M < N < P
assert is_prime(P), f'{P} is not prime'
assert is_pow2(N), f'{N} is not a power of 2'

# TODO: plot
L = fri.domain(w, N, P)
p = polynomial.interp(L[:M], ys, lambda x: F(x, P))
print(f"Message length = {M}")
print(f"Polynomial degree = {p.degree()}")

### Commit ###
iop = iop.Writer()
prover = fri.Prover(
    N = N,
    P = P,
    w = w,
    iop = iop,
    exp_factor = EXP_FACTOR
)

prover.commit(fri.eval_poly(p, fri.domain(w, N, P), P))

print("codewords", prover.codewords)

### Query ###
verifier = fri.Verifier(
    N = N,
    P = P,
    w = w,
    challenges = iop.challenges,
    merkle_roots = iop.merkle_roots,
    exp_factor = EXP_FACTOR
)

# Random indexes without replacement
NUM_QUERIES = 2
assert NUM_QUERIES <= N

rands = [int(r) for r in np.random.choice(range(N), size=NUM_QUERIES, replace=False)]
for i in rands:
    (vals, proofs, codeword) = prover.prove(i)
    verifier.verify(i, vals, proofs, codeword)


ys: [1, 0, 1, 0]
Message length = 4
Polynomial degree = 3
codewords [[1, 0, 1, 0, 61, 217, 54, 308, 190, 238, 188, 68, 169, 303, 178, 45], [224, 218, 45, 168, 176, 182, 18, 232], [137, 137, 137, 137]]


In [1]:
from fft import fft_rec, fft, ifft, eval_poly
P = 17
N = 16
g = 3

# Multiplicative group
gs = [pow(g, i, P) for i in range(P - 1)]
assert len(set(gs)) == P - 1

# N-th primitive root
w = pow(g, (P - 1)//N, P)
print("w", w)
# Check w is a N-th primitive root
assert pow(w, N, P) == 1
assert pow(w, N//2, P) == -1 % P

# FFT evaluation domain
ws = [pow(w, i, P) for i in range(N)]
# Example polynomial
f = [i + 1 for i in range(N)]
print("f", f)
print("ws", ws)

e = eval_poly(f, ws, P)
r = fft_rec(f, ws, P)
l = fft(f, ws, P)
i = ifft(l, ws, P)

print("e", e)
print("r", r)
print("l", l)
print("i", i)
assert r == e
assert l == e
assert i == f

# Evaluate a polynomial on a domain larger than the degree of the polynomial
f = [i + 1 for i in range(N//2)] + ([0] * (N//2))
print("f", f)

ys = fft(f, ws, P)
print("ys", ys)

cs = ifft(ys, ws, P)
print("cs", cs)

assert cs == f

w 3
f [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
ws [1, 3, 9, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6]
e [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
r [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
l [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
i [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
f [1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0]
ys [2, 5, 1, 9, 12, 13, 3, 5, 13, 0, 6, 11, 14, 8, 8, 8]
cs [1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0]


In [2]:
# N-th root of unity
# Any number w such that w^N = 1 mod P

# Primitive N-th root of unity
# w^N = 1 and w^k != 1 for 0 < k < N 

# Steps to find the N-th primitive root of unity mod P
# 0. Pick a prime P -> This defines the finite field F[P]
#    TODO: Why prime?
#    - nonzero element has a multiplicative inverse
# 1. Consider the multiplicative group F[P]x = {1,2,3,...,P−1} 
# 2. Find a generator g of F[P]x
# 3. Find N = largest power of 2 dividing P−1
#    The subgroup of order N is:
#      H = { g^((P−1)/N)^i  for i = 0 ... N−1 }
# 4. To find a primitive N-th root of unity:
#    Let w = g^((P−1)/N) (g^(P-1) = 1 (Fermat's little theorem))
#    Verify:
#      - w^N = 1
#      TODO: why this check suffices
#      - w^(N/2) = -1

from field import find_generator
from utils import find_largest_pow_2

# 0. Pick a prime P
# Starkware prime
P = 2**251 + 17 * 2**192 + 1
# P = 17

# 2. Find a generator g
g = find_generator(P)
print("g = ", g)

# 3. Find N = largest power of 2 dividing P - 1
k = find_largest_pow_2(P - 1)
N = 2**k
print("k =", k)
print("n =", N)

w = pow(g, ((P - 1) // N), P)

assert pow(w, N, P) == 1
assert pow(w, N // 2, P) == (-1 % P)

print(f"{N} th primitive root of unity =", w)

g =  3
k = 192
n = 6277101735386680763835789423207666416102355444464034512896
6277101735386680763835789423207666416102355444464034512896 th primitive root of unity = 145784604816374866144131285430889962727208297722245411306711449302875041684


In [2]:
[0] * -1

[]

In [3]:
[0] + [1] * 3

[0, 1, 1, 1]